# TODO:

- Remove references
- reach goal: handle chunking witth tables
- handle footers</p>


# Set up notebook


In [1]:
# Warning control
import warnings

warnings.filterwarnings("ignore")
import logging

# logger = logging.getLogger()
# logger.setLevel(logging.CRITICAL)

import nltk

nltk.download("punkt_tab")
nltk.download("averaged_perceptron_tagger_eng")
import collections
from io import StringIO
from lxml import etree

[nltk_data] Downloading package punkt_tab to /Users/ducdo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/ducdo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [4]:
import json
from IPython.display import JSON

from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.chunking.basic import chunk_elements
from unstructured.chunking.title import chunk_by_title


from unstructured.partition.pdf import partition_pdf, partition_pdf_or_image
from unstructured.staging.base import dict_to_elements
from unstructured_client.models import operations
import chromadb

from pathlib import Path
import pickle
import os
from dotenv import load_dotenv
from IPython.core.display import HTML


In [5]:
import sys

sys.path.append(str(Path.cwd().parents[1]))
from backend.config import CONNECTION_STRING, COLLECTION_NAME


In [6]:
load_dotenv()
UNSTRUCTURED_API_KEY = os.getenv("UNSTRUCTURED_API_KEY")

In [7]:
import unstructured_client


client = unstructured_client.UnstructuredClient(
    api_key_auth=UNSTRUCTURED_API_KEY,
    server_url="https://api.unstructuredapp.io",
)


# Testing with xml


In [18]:
xmlString = """
<!DOCTYPE article
PUBLIC "-//NLM//DTD JATS (Z39.96) Journal Archiving and Interchange DTD with MathML3 v1.3 20210610//EN" "JATS-archivearticle1-3-mathml3.dtd">
<article dtd-version="1.3" xml:lang="en" article-type="case-report"><?properties open_access?><processing-meta base-tagset="archiving" mathml-version="3.0" table-model="xhtml" tagset-family="jats"><restricted-by>pmc</restricted-by></processing-meta><front><journal-meta><journal-id journal-id-type="nlm-ta">J Fungi (Basel)</journal-id><journal-id journal-id-type="iso-abbrev">J Fungi (Basel)</journal-id><journal-id journal-id-type="publisher-id">jof</journal-id><journal-title-group><journal-title>Journal of Fungi</journal-title></journal-title-group><issn pub-type="epub">2309-608X</issn><publisher><publisher-name>MDPI</publisher-name></publisher></journal-meta><article-meta><article-id pub-id-type="pmid">34575746</article-id><article-id pub-id-type="pmc">8472188</article-id><article-id pub-id-type="doi">10.3390/jof7090708</article-id><article-id pub-id-type="publisher-id">jof-07-00708</article-id><article-categories><subj-group subj-group-type="heading"><subject>Case Report</subject></subj-group></article-categories><title-group><article-title>Mucormycosis in CAPA, a Possible Fungal Super-Infection</article-title></title-group><contrib-group><contrib contrib-type="author"><name><surname>Saltini</surname><given-names>Paola</given-names></name><xref rid="af1-jof-07-00708" ref-type="aff">1</xref><xref rid="af2-jof-07-00708" ref-type="aff">2</xref><xref rid="af3-jof-07-00708" ref-type="aff">3</xref><xref rid="fn1-jof-07-00708" ref-type="author-notes">&#x02020;</xref></contrib><contrib contrib-type="author"><contrib-id contrib-id-type="orcid" authenticated="true">https://orcid.org/0000-0003-1386-6180</contrib-id><name><surname>Palomba</surname><given-names>Emanuele</given-names></name><xref rid="af1-jof-07-00708" ref-type="aff">1</xref><xref rid="af2-jof-07-00708" ref-type="aff">2</xref><xref rid="af3-jof-07-00708" ref-type="aff">3</xref><xref rid="fn1-jof-07-00708" ref-type="author-notes">&#x02020;</xref></contrib><contrib contrib-type="author"><name><surname>Castelli</surname><given-names>Valeria</given-names></name><xref rid="af1-jof-07-00708" ref-type="aff">1</xref><xref rid="af2-jof-07-00708" ref-type="aff">2</xref><xref rid="af3-jof-07-00708" ref-type="aff">3</xref></contrib><contrib contrib-type="author"><name><surname>Fava</surname><given-names>Marco</given-names></name><xref rid="af1-jof-07-00708" ref-type="aff">1</xref><xref rid="af2-jof-07-00708" ref-type="aff">2</xref><xref rid="af3-jof-07-00708" ref-type="aff">3</xref></contrib><contrib contrib-type="author"><name><surname>Alagna</surname><given-names>Laura</given-names></name><xref rid="af1-jof-07-00708" ref-type="aff">1</xref></contrib><contrib contrib-type="author"><contrib-id contrib-id-type="orcid" authenticated="true">https://orcid.org/0000-0003-4988-0427</contrib-id><name><surname>Biscarini</surname><given-names>Simona</given-names></name><xref rid="af1-jof-07-00708" ref-type="aff">1</xref></contrib><contrib contrib-type="author"><name><surname>Mantero</surname><given-names>Marco</given-names></name><xref rid="af2-jof-07-00708" ref-type="aff">2</xref><xref rid="af4-jof-07-00708" ref-type="aff">4</xref></contrib><contrib contrib-type="author"><contrib-id contrib-id-type="orcid" authenticated="true">https://orcid.org/0000-0002-2285-9970</contrib-id><name><surname>Blasi</surname><given-names>Francesco</given-names></name><xref rid="af2-jof-07-00708" ref-type="aff">2</xref><xref rid="af4-jof-07-00708" ref-type="aff">4</xref></contrib><contrib contrib-type="author"><name><surname>Grancini</surname><given-names>Anna</given-names></name><xref rid="af5-jof-07-00708" ref-type="aff">5</xref></contrib><contrib contrib-type="author"><name><surname>Bandera</surname><given-names>Alessandra</given-names></name><xref rid="af1-jof-07-00708" ref-type="aff">1</xref><xref rid="af2-jof-07-00708" ref-type="aff">2</xref><xref rid="af3-jof-07-00708" ref-type="aff">3</xref></contrib><contrib contrib-type="author"><name><surname>Gori</surname><given-names>Andrea</given-names></name><xref rid="af1-jof-07-00708" ref-type="aff">1</xref><xref rid="af2-jof-07-00708" ref-type="aff">2</xref><xref rid="af3-jof-07-00708" ref-type="aff">3</xref></contrib><contrib contrib-type="author"><name><surname>Muscatello</surname><given-names>Antonio</given-names></name><xref rid="af1-jof-07-00708" ref-type="aff">1</xref></contrib><contrib contrib-type="author"><contrib-id contrib-id-type="orcid" authenticated="true">https://orcid.org/0000-0002-0383-9579</contrib-id><name><surname>Lombardi</surname><given-names>Andrea</given-names></name><xref rid="af1-jof-07-00708" ref-type="aff">1</xref><xref rid="af2-jof-07-00708" ref-type="aff">2</xref><xref rid="c1-jof-07-00708" ref-type="corresp">*</xref></contrib></contrib-group><contrib-group><contrib contrib-type="editor"><name><surname>Perlin</surname><given-names>David S.</given-names></name><role>Academic Editor</role></contrib></contrib-group><aff id="af1-jof-07-00708"><label>1</label>Infectious Diseases Unit, Fondazione IRCCS Ca&#x02019; Granda Ospedale Maggiore Policlinico, 20122 Milano, Italy; <email>paola.saltini@unimi.it</email> (P.S.); <email>emanuele.palomba@unimi.it</email> (E.P.); <email>valeria.castelli@unimi.it</email> (V.C.); <email>marco.fava@unimi.it</email> (M.F.); <email>laura.alagna@policlinico.mi.it</email> (L.A.); <email>simona.biscarini@policlinico.mi.it</email> (S.B.); <email>alessandra.bandera@unimi.it</email> (A.B.); <email>andrea.gori@unimi.it</email> (A.G.); <email>antonio.muscatello@policlinico.mi.it</email> (A.M.)</aff><aff id="af2-jof-07-00708"><label>2</label>Department of Pathophysiology and Transplantation, University of Milano, 20122 Milano, Italy; <email>marco.mantero@unimi.it</email> (M.M.); <email>francesco.blasi@unimi.it</email> (F.B.)</aff><aff id="af3-jof-07-00708"><label>3</label>Centre for Multidisciplinary Research in Health Science (MACH), University of Milano, 20122 Milano, Italy</aff><aff id="af4-jof-07-00708"><label>4</label>Respiratory Unit and Cystic Fibrosis Adult Center, IRCCS Fondazione IRCCS Ca&#x02019; Granda Ospedale Maggiore Policlinico, 20122 Milano, Italy</aff><aff id="af5-jof-07-00708"><label>5</label>UOS Microbiology, Central Laboratory, IRCCS Foundation, C&#x000e0; Granda Ospedale Maggiore Policlinico, 20122 Milano, Italy; <email>anna.grancini@policlinico.mi.it</email></aff><author-notes><corresp id="c1-jof-07-00708"><label>*</label>Correspondence: <email>andrea.lombardi@unimi.it</email>; Tel.: +39-02-5503-4767</corresp><fn id="fn1-jof-07-00708"><label>&#x02020;</label><p>These authors contributed equally to the study.</p></fn></author-notes><pub-date pub-type="epub"><day>28</day><month>8</month><year>2021</year></pub-date><pub-date pub-type="collection"><month>9</month><year>2021</year></pub-date><volume>7</volume><issue>9</issue><elocation-id>708</elocation-id><history><date date-type="received"><day>06</day><month>8</month><year>2021</year></date><date date-type="accepted"><day>26</day><month>8</month><year>2021</year></date></history><permissions><copyright-statement>&#x000a9; 2021 by the authors.</copyright-statement><copyright-year>2021</copyright-year><license><ali:license_ref xmlns:ali="http://www.niso.org/schemas/ali/1.0/" specific-use="textmining" content-type="ccbylicense">https://creativecommons.org/licenses/by/4.0/</ali:license_ref><license-p>Licensee MDPI, Basel, Switzerland. This article is an open access article distributed under the terms and conditions of the Creative Commons Attribution (CC BY) license (<ext-link xmlns:xlink="http://www.w3.org/1999/xlink" ext-link-type="uri" xlink:href="https://creativecommons.org/licenses/by/4.0/">https://creativecommons.org/licenses/by/4.0/</ext-link>).</license-p></license></permissions><abstract><p>The occurrence of pulmonary fungal superinfection due to <italic toggle="yes">Aspergillus</italic> spp. in patients with COVID-19 is a well-described complication associated with significant morbidity and mortality. This can be related to a directed effect of the virus and to the immunosuppressive role of the therapies administered for the disease. Here, we describe the first case of pulmonary infection due to Mucorales occurring in a patient with a concomitant diagnosis of COVID-19-associated pulmonary aspergillosis.</p></abstract><kwd-group><kwd>mucormycosis</kwd><kwd>CAPA</kwd><kwd>COVID-19</kwd></kwd-group></article-meta></front><body><sec sec-type="intro" id="sec1-jof-07-00708"><title>1. Introduction</title><p>Since December 2019, the coronavirus disease 2019 (COVID-19) quickly spread worldwide and was declared as a pandemic by the World Health Organization (WHO) on 11 March 2020 [<xref rid="B1-jof-07-00708" ref-type="bibr">1</xref>]. The disease pattern of COVID-19 can range from mild to life-threatening pneumonia, with worst outcomes in elder patients and people with comorbidities such as severe asthma, diabetes mellitus or cancer [<xref rid="B2-jof-07-00708" ref-type="bibr">2</xref>]. The direct damage of the airway epithelium caused by the virus, the severe lymphopenia and the immune dysfunction induced by the infection [<xref rid="B3-jof-07-00708" ref-type="bibr">3</xref>,<xref rid="B4-jof-07-00708" ref-type="bibr">4</xref>] facilitate bacterial and fungal superinfection. Moreover, patients requiring oxygen or ventilatory support are currently managed with systemic glucocorticoids, the only treatment which has been demonstrated to reduce mortality [<xref rid="B5-jof-07-00708" ref-type="bibr">5</xref>,<xref rid="B6-jof-07-00708" ref-type="bibr">6</xref>], but steroid administration is a well-known risk factor for infections [<xref rid="B7-jof-07-00708" ref-type="bibr">7</xref>]. Usually, steroid therapy is administered as dexamethasone 6 mg/day based on the results of the RECOVERY trial [<xref rid="B8-jof-07-00708" ref-type="bibr">8</xref>], but other immunomodulatory molecules with immunosuppressive action are widely used, even if the efficacy of methylprednisolone and hydrocortisone has not been so clearly demonstrated [<xref rid="B9-jof-07-00708" ref-type="bibr">9</xref>,<xref rid="B10-jof-07-00708" ref-type="bibr">10</xref>,<xref rid="B11-jof-07-00708" ref-type="bibr">11</xref>]. Opportunistic fungal infections are of concern in such patients as an additional contributing factor to mortality.</p><p>There have been several reports of COVID-19-associated pulmonary aspergillosis (CAPA) [<xref rid="B12-jof-07-00708" ref-type="bibr">12</xref>] and some cases of mucormycosis, especially in patients with diabetes [<xref rid="B13-jof-07-00708" ref-type="bibr">13</xref>,<xref rid="B14-jof-07-00708" ref-type="bibr">14</xref>]. The diagnosis of COVID-19-associated mucormycosis (CAM) is more challenging because it is less frequently suspected by the clinicians and the causative agents are more difficult to isolate in cultures. Furthermore, the diagnosis cannot be helped by biomarkers such as &#x003b2;-D-glucan and galactomannan, as it is possible for aspergillosis [<xref rid="B15-jof-07-00708" ref-type="bibr">15</xref>,<xref rid="B16-jof-07-00708" ref-type="bibr">16</xref>].</p><p>Here we report the first case of the coexistence of two destructive fungal diseases, pulmonary aspergillosis and mucormycosis, in a patient with COVID-19 who was treated with corticosteroids. Our case emphasizes the importance of the clinical suspicion of invasive mould disease in this setting and the need to repeat microbiological investigations on bronchoalveolar lavage or other respiratory specimens, especially in case of poor response to treatment.</p></sec><sec id="sec2-jof-07-00708"><title>2. Materials and Methods</title><p>In our laboratory, respiratory samples of patients at risk for fungal infections, such as patients hospitalized in the intensive care unit, are routinely investigated for the presence of fungi. Our protocols include the culture of respiratory samples on Sabouraud Dextrose Agar, incubated at 32 &#x000b0;C for 5&#x02013;20 days.</p><sec><title>Genus and Species Identification and Susceptibility Testing</title><p>Identification to genus and species level is strongly recommended for improved epidemiological data and to guide the choice of the antifungal treatment. We employed matrix assisted laser desorption ionisation time of flight (MALDI-TOF; Vitek MS BioM&#x000e8;rieux&#x02013;IVD; Liyon, France) together with the macro and microscopic morphological evaluation of the fungus. The commercial methods E-test (BioM&#x000e8;rieux) with RPMI-1640 medium (BioM&#x000e8;rieux) was used for the antifungal susceptibility. For Aspergillus spp., the EUCAST clinical breakpoint for fungi was employed [<xref rid="B17-jof-07-00708" ref-type="bibr">17</xref>]. No interpretative clinical breakpoints are available for Mucorales spp., and the classification of isolates as susceptible or resistant is not possible.</p></sec></sec><sec id="sec3-jof-07-00708"><title>3. Case Presentation</title><p>A 72-year-old man with a history of myelodysplastic syndrome, type 2 diabetes and arterial hypertension was admitted to the Respiratory Care unit of our hospital with a diagnosis of SARS-CoV-2 pneumonia. The patient developed acute respiratory distress. On day 5 he underwent non-invasive ventilation with Continuous Positive Airway Pressure (CPAP) and a treatment with intravenous corticosteroids, initially with dexamethasone 6 mg every 24 h (two-day course) and then with methylprednisolone 40 mg every 12 h (25-days course with subsequent decalage). Given the sustained fever, the altered biochemistry indexes (C reactive protein 31.65 mg/dL) and his myelosuppression (neutrophil count 1270 /mmc), the patient was treated with multiple empiric antibiotic lines (piperacillin/tazobactam, cefepime and ceftobiprole). Finally, on day 27, he was treated with daptomycin and ceftaroline for <italic toggle="yes">Enterococcus faecium</italic> bloodstream infection, with a subsequent shift to vancomycin and ceftaroline for the lack of clinical response. On day 30, a transthoracic echocardiography was performed, excluding valvular vegetations compatible with endocarditis, whereas the thoracic high-resolution computed tomography (HRCT) performed on day 38 showed pseudo-nodular cavitary consolidations in the right apical zone (<xref rid="jof-07-00708-f001" ref-type="fig">Figure 1</xref>A). The CT scan of facial bones only showed an inflammatory thickness of the left maxillary sinus. In the suspect of COVID-19 associated pulmonary aspergillosis (CAPA), voriconazole iv (300 mg every 12 h) was started. <italic toggle="yes">Aspergillus fumigatus</italic> susceptible to azoles (the MIC of voriconazole was 0.094 mg/L) was isolated on samples of broncho-alveolar lavage performed on day 39, and antibiotic therapy with vancomycin and ceftaroline was stopped. Due to the lack of clinical improvement (the need for oxygen therapy with high-flow nasal cannula and the persistent fever), on day 53, therapy was then maximized, adding anidulafungin (200 mg the first day, then 100 mg/day) to voriconazole and introducing wide range empiric antibiotic therapy with linezolid (600 mg every 12 h) and piperacillin/tazobactam (3.50 g every 8 h).</p><p>The patient was then transferred to the Infectious Diseases (ID) department because of the worsening pulmonary radiological pattern and the persistence of fever. On day 55, the patient underwent a fundus oculi exam, which resulted negative for local infections. Blood cultures were performed at different times during his ID ward stay, and they were all negative. Blood cultures for mycobacteria and blood samples for galactomannan, &#x003b2;-D-glucan, HSV-1 DNA, HSV-2 DNA, CMV-DNA, EBV-DNA and BKV- DNA, serologies for <italic toggle="yes">Toxoplasma gondii</italic> and <italic toggle="yes">Strongyloides stercoralis</italic> and bacteria associated with atypical pneumonia were also collected and yielded a negative result. On day 58, the patient underwent a bone marrow biopsy and bone marrow aspiration, tested negative for CMV, EBV and <italic toggle="yes">Parvovirus</italic> B-19 DNA, <italic toggle="yes">Leishmania</italic> spp. PCR and bacterial and mycobacterial cultures. The same day, gas exchange deteriorated and respiratory distress increased. Non-invasive ventilation with CPAP was then restarted. A thorax CT scan showed a ground-glass and crazy-paving pattern (<xref rid="jof-07-00708-f001" ref-type="fig">Figure 1</xref>C). Therapy with methylprednisolone and trimethoprim-sulfamethoxazole (STX) was then started due to the clinical and radiological suspicion of <italic toggle="yes">Pneumocystis jirovecii</italic> pneumonia, despite the ongoing prophylaxis with atovaquone that was started because of myelodysplastic syndrome and corticosteroids therapy. Moreover, linezolid was stopped and meropenem was substituted with piperacillin/tazobactam. On day 59, a bronchial aspiration (BAS) was performed, and a Mucorales (identified after some weeks as a <italic toggle="yes">Mucor circinelloides</italic> resistant to voriconazole and susceptible to amphotericin B with a MIC of 0.094 mg/L) was identified. On day 61, the antifungal therapy was changed and the patient was administered with amphotericin B (7 mg/kg/day) and caspofungin (50 mg/day). The search for <italic toggle="yes">Pneumocystis jirovecii</italic> DNA on BAS material was negative; therefore, STX was stopped and steroidal therapy was reduced. The patient further deteriorated, with worsening hypoxaemia and dyspnoea despite maximized CPAP support, and he finally died on day 64.</p></sec><sec sec-type="discussion" id="sec4-jof-07-00708"><title>4. Discussion</title><p>Different incidence rates of CAPA have been described in the literature, from 0.7&#x02013;7.7% among COVID patients to higher percentages among ICU patients and people with severe COVID-19 pneumonia [<xref rid="B12-jof-07-00708" ref-type="bibr">12</xref>,<xref rid="B18-jof-07-00708" ref-type="bibr">18</xref>]. The more frequent baseline conditions associated with CAPA are diabetes mellitus, corticosteroid administration, chronic cardiovascular diseases, renal failure and obesity [<xref rid="B12-jof-07-00708" ref-type="bibr">12</xref>,<xref rid="B19-jof-07-00708" ref-type="bibr">19</xref>]. As highlighted by our case, the clinical management of fragile patients with COVID-19 represents a difficult challenge. The coexistence of an impaired immune system alongside comorbidities often hampers differential diagnosis and the correct clinical and therapeutic approach. In this setting, SARS-CoV-2 infection can be the initial trigger of respiratory failure, and it favors infective complications. This can be related to a direct effect, consequently to airway epithelium damage and gut microbiota composition alteration, which favors fungal superinfection. Or, it can happen indirectly due to prolonged hospitalization, ventilatory support and the long courses of corticosteroid therapy required in the management of COVID-19 patients [<xref rid="B20-jof-07-00708" ref-type="bibr">20</xref>]. For these reasons, it is crucial to be aware of such risks, monitoring these patients with even further precaution and eventually performing regular screening for opportunistic infections (e.g., fungal biomarkers such as &#x003b2;-D-glucan and galactomannan). Furthermore, in the case of unresponsiveness to standard care for COVID-19, the clinical workup should be extended to extensive cultural enquiries (e.g., bronchoalveolar lavage) in addition to thoracic and paranasal sinuses imaging, aiming at the early recognition of microbiological and radiological signs of fungal disease. In conclusion, since SARS-CoV-2 continues to be a worldwide concern, other data are needed to better assess the incidence and the management of fungal superinfections in COVID-19. Prompt diagnosis is paramount to start appropriate antifungal therapy and, as emphasized by our experience, shape the clinical course of fragile COVID-19 patients.</p></sec></body><back><fn-group><fn><p><bold>Publisher&#x02019;s Note:</bold> MDPI stays neutral with regard to jurisdictional claims in published maps and institutional affiliations.</p></fn></fn-group><notes><title>Author Contributions</title><p>Resources, data curation and writing, P.S., E.P. and A.L.; A.G. (Anna Grancini) performed microbiological tests; all other authors participated in the review and editing of the final graft. All authors have read and agreed to the published version of the manuscript.</p></notes><notes><title>Funding</title><p>This research received no external funding.</p></notes><notes notes-type="data-availability"><title>Data Availability Statement</title><p>Data will be provided on reasonable request.</p></notes><notes notes-type="COI-statement"><title>Conflicts of Interest</title><p>The authors declare no conflict of interest.</p></notes><ref-list><title>References</title><ref id="B1-jof-07-00708"><label>1.</label><element-citation publication-type="webpage"><article-title>WHO Director-General&#x02019;s Opening Remarks at the Media Briefing on COVID-19&#x02014;11 March 2020</article-title><comment>Available online: <ext-link xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="https://www.who.int/dg/speeches/detail/who-director-general-s-opening-remarks-at-the-media-briefing-on-covid-19---11-march-2020" ext-link-type="uri">https://www.who.int/dg/speeches/detail/who-director-general-s-opening-remarks-at-the-media-briefing-on-covid-19---11-march-2020</ext-link></comment><date-in-citation content-type="access-date" iso-8601-date="2020-03-17">(accessed on 17 March 2020)</date-in-citation></element-citation></ref><ref id="B2-jof-07-00708"><label>2.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Williamson</surname><given-names>E.J.</given-names></name>
<name><surname>Walker</surname><given-names>A.J.</given-names></name>
<name><surname>Bhaskaran</surname><given-names>K.</given-names></name>
<name><surname>Bacon</surname><given-names>S.</given-names></name>
<name><surname>Bates</surname><given-names>C.</given-names></name>
<name><surname>Morton</surname><given-names>C.E.</given-names></name>
<name><surname>Curtis</surname><given-names>H.J.</given-names></name>
<name><surname>Mehrkar</surname><given-names>A.</given-names></name>
<name><surname>Evans</surname><given-names>D.</given-names></name>
<name><surname>Inglesby</surname><given-names>P.</given-names></name>
<etal/>
</person-group><article-title>Factors associated with COVID-19-related death using OpenSAFELY</article-title><source>Nature</source><year>2020</year><volume>584</volume><fpage>430</fpage><lpage>436</lpage><pub-id pub-id-type="doi">10.1038/s41586-020-2521-4</pub-id><pub-id pub-id-type="pmid">32640463</pub-id></element-citation></ref><ref id="B3-jof-07-00708"><label>3.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Zhang</surname><given-names>Y.</given-names></name>
<name><surname>Geng</surname><given-names>X.</given-names></name>
<name><surname>Tan</surname><given-names>Y.</given-names></name>
<name><surname>Li</surname><given-names>Q.</given-names></name>
<name><surname>Xu</surname><given-names>C.</given-names></name>
<name><surname>Xu</surname><given-names>J.</given-names></name>
<name><surname>Hao</surname><given-names>L.</given-names></name>
<name><surname>Zeng</surname><given-names>Z.</given-names></name>
<name><surname>Luo</surname><given-names>X.</given-names></name>
<name><surname>Liu</surname><given-names>F.</given-names></name>
<etal/>
</person-group><article-title>New understanding of the damage of SARS-CoV-2 infection outside the respiratory system</article-title><source>Biomed. Pharm.</source><year>2020</year><volume>127</volume><fpage>110195</fpage><pub-id pub-id-type="doi">10.1016/j.biopha.2020.110195</pub-id><pub-id pub-id-type="pmid">32361161</pub-id></element-citation></ref><ref id="B4-jof-07-00708"><label>4.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Qin</surname><given-names>C.</given-names></name>
<name><surname>Zhou</surname><given-names>L.</given-names></name>
<name><surname>Hu</surname><given-names>Z.</given-names></name>
<name><surname>Zhang</surname><given-names>S.</given-names></name>
<name><surname>Yang</surname><given-names>S.</given-names></name>
<name><surname>Tao</surname><given-names>Y.</given-names></name>
<name><surname>Xie</surname><given-names>C.</given-names></name>
<name><surname>Ma</surname><given-names>K.</given-names></name>
<name><surname>Shang</surname><given-names>K.</given-names></name>
<name><surname>Wang</surname><given-names>W.</given-names></name>
<etal/>
</person-group><article-title>Dysregulation of immune response in patients with coronavirus 2019 (COVID-19) in Wuhan, China</article-title><source>Clin. Infect. Dis.</source><year>2020</year><volume>71</volume><fpage>762</fpage><lpage>768</lpage><pub-id pub-id-type="doi">10.1093/cid/ciaa248</pub-id><pub-id pub-id-type="pmid">32161940</pub-id></element-citation></ref><ref id="B5-jof-07-00708"><label>5.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Sterne</surname><given-names>J.</given-names></name>
<name><surname>Murthy</surname><given-names>S.</given-names></name>
<name><surname>Diaz</surname><given-names>J.V.</given-names></name>
<name><surname>Slutsky</surname><given-names>A.S.</given-names></name>
<name><surname>Villar</surname><given-names>J.</given-names></name>
<name><surname>Angus</surname><given-names>D.C.</given-names></name>
<name><surname>Annane</surname><given-names>D.</given-names></name>
<name><surname>Azevedo</surname><given-names>L.C.P.</given-names></name>
<name><surname>Berwanger</surname><given-names>O.</given-names></name>
<name><surname>Cavalcanti</surname><given-names>A.B.</given-names></name>
<etal/>
</person-group><article-title>Association between Administration of Systemic Corticosteroids and Mortality among Critically Ill Patients with COVID-19, A Meta-analysis</article-title><source>JAMA&#x02014;J. Am. Med. Assoc.</source><year>2020</year><volume>324</volume><fpage>1330</fpage><lpage>1341</lpage><pub-id pub-id-type="doi">10.1001/jama.2020.17023</pub-id></element-citation></ref><ref id="B6-jof-07-00708"><label>6.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Siemieniuk</surname><given-names>R.A.</given-names></name>
<name><surname>Bartoszko</surname><given-names>J.J.</given-names></name>
<name><surname>Ge</surname><given-names>L.</given-names></name>
<name><surname>Zeraatkar</surname><given-names>D.</given-names></name>
<name><surname>Izcovich</surname><given-names>A.</given-names></name>
<name><surname>Kum</surname><given-names>E.</given-names></name>
<name><surname>Pardo-Hernandez</surname><given-names>H.</given-names></name>
<name><surname>Qasim</surname><given-names>A.</given-names></name>
<name><surname>Martinez</surname><given-names>J.P.D.</given-names></name>
<name><surname>Rochwerg</surname><given-names>B.</given-names></name>
<etal/>
</person-group><article-title>Drug treatments for COVID-19, living systematic review and network meta-analysis</article-title><source>BMJ</source><year>2020</year><volume>370</volume><fpage>m2980</fpage><pub-id pub-id-type="doi">10.1136/bmj.m2980</pub-id><pub-id pub-id-type="pmid">32732190</pub-id></element-citation></ref><ref id="B7-jof-07-00708"><label>7.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Fardet</surname><given-names>L.</given-names></name>
<name><surname>Petersen</surname><given-names>I.</given-names></name>
<name><surname>Nazareth</surname><given-names>I.</given-names></name>
</person-group><article-title>Common Infections in Patients Prescribed Systemic Glucocorticoids in Primary Care: A Population-Based Cohort Study</article-title><source>PLoS Med.</source><year>2016</year><volume>13</volume><elocation-id>e1002024</elocation-id><pub-id pub-id-type="doi">10.1371/journal.pmed.1002024</pub-id><pub-id pub-id-type="pmid">27218256</pub-id></element-citation></ref><ref id="B8-jof-07-00708"><label>8.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<collab>RECOVERY Collaborative Group</collab>
<name><surname>Horby</surname><given-names>P.</given-names></name>
<name><surname>Lim</surname><given-names>W.S.</given-names></name>
<name><surname>Emberson</surname><given-names>J.R.</given-names></name>
<name><surname>Mafham</surname><given-names>M.</given-names></name>
<name><surname>Bell</surname><given-names>J.L.</given-names></name>
<name><surname>Linsell</surname><given-names>L.</given-names></name>
<name><surname>Staplin</surname><given-names>N.</given-names></name>
<name><surname>Brightling</surname><given-names>C.</given-names></name>
<name><surname>Ustianowski</surname><given-names>A.</given-names></name>
<etal/>
</person-group><article-title>Dexamethasone in Hospitalized Patients with COVID-19</article-title><source>N. Engl. J. Med.</source><year>2021</year><volume>384</volume><fpage>693</fpage><lpage>704</lpage><pub-id pub-id-type="doi">10.1056/nejmoa2021436</pub-id><pub-id pub-id-type="pmid">32678530</pub-id></element-citation></ref><ref id="B9-jof-07-00708"><label>9.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Dequin</surname><given-names>P.-F.</given-names></name>
<name><surname>Heming</surname><given-names>N.</given-names></name>
<name><surname>Meziani</surname><given-names>F.</given-names></name>
<name><surname>Plantef&#x000e8;ve</surname><given-names>G.</given-names></name>
<name><surname>Voiriot</surname><given-names>G.</given-names></name>
<name><surname>Badi&#x000e9;</surname><given-names>J.</given-names></name>
<name><surname>Fran&#x000e7;ois</surname><given-names>B.</given-names></name>
<name><surname>Aubron</surname><given-names>C.</given-names></name>
<name><surname>Ricard</surname><given-names>J.-D.</given-names></name>
<name><surname>Ehrmann</surname><given-names>S.</given-names></name>
<etal/>
</person-group><article-title>Effect of Hydrocortisone on 21-Day Mortality or Respiratory Support among Critically Ill Patients with COVID-19, A Randomized Clinical Trial</article-title><source>JAMA&#x02014;J. Am. Med. Assoc.</source><year>2020</year><volume>324</volume><fpage>1298</fpage><lpage>1306</lpage><pub-id pub-id-type="doi">10.1001/jama.2020.16761</pub-id><pub-id pub-id-type="pmid">32876689</pub-id></element-citation></ref><ref id="B10-jof-07-00708"><label>10.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Angus</surname><given-names>D.C.</given-names></name>
<name><surname>Derde</surname><given-names>L.</given-names></name>
<name><surname>Al-Beidh</surname><given-names>F.</given-names></name>
<name><surname>Annane</surname><given-names>D.</given-names></name>
<name><surname>Arabi</surname><given-names>Y.</given-names></name>
<name><surname>Beane</surname><given-names>A.</given-names></name>
<name><surname>Van Bentum-Puijk</surname><given-names>W.</given-names></name>
<name><surname>Berry</surname><given-names>L.</given-names></name>
<name><surname>Bhimani</surname><given-names>Z.</given-names></name>
<name><surname>Bonten</surname><given-names>M.</given-names></name>
<etal/>
</person-group><article-title>Effect of Hydrocortisone on Mortality and Organ Support in Patients with Severe COVID-19, the REMAP-CAP COVID-19 Corticosteroid Domain Randomized Clinical Trial</article-title><source>JAMA&#x02014;J. Am. Med. Assoc.</source><year>2020</year><volume>324</volume><fpage>1317</fpage><lpage>1329</lpage><pub-id pub-id-type="doi">10.1001/jama.2020.17022</pub-id></element-citation></ref><ref id="B11-jof-07-00708"><label>11.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Jeronimo</surname><given-names>C.M.P.</given-names></name>
<name><surname>Farias</surname><given-names>M.E.L.</given-names></name>
<name><surname>Val</surname><given-names>F.F.A.</given-names></name>
<name><surname>Sampaio</surname><given-names>V.S.</given-names></name>
<name><surname>Alexandre</surname><given-names>M.A.A.</given-names></name>
<name><surname>Melo</surname><given-names>G.C.</given-names></name>
<name><surname>Safe</surname><given-names>I.P.</given-names></name>
<name><surname>Borba</surname><given-names>M.G.S.</given-names></name>
<name><surname>Netto</surname><given-names>R.L.A.</given-names></name>
<name><surname>Maciel</surname><given-names>A.B.S.</given-names></name>
<etal/>
</person-group><article-title>Methylprednisolone as Adjunctive Therapy for Patients Hospitalized with Coronavirus Disease 2019 (COVID-19; Metcovid): A Randomized, Double-blind, Phase IIb, Placebo-controlled Trial</article-title><source>Clin. Infect. Dis.</source><year>2021</year><volume>72</volume><fpage>e373</fpage><lpage>e381</lpage><pub-id pub-id-type="doi">10.1093/cid/ciaa1177</pub-id><pub-id pub-id-type="pmid">32785710</pub-id></element-citation></ref><ref id="B12-jof-07-00708"><label>12.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Salmanton-Garcia</surname><given-names>J.</given-names></name>
<name><surname>Sprute</surname><given-names>R.</given-names></name>
<name><surname>Stemler</surname><given-names>J.</given-names></name>
<name><surname>Bartoletti</surname><given-names>M.</given-names></name>
<name><surname>Dupont</surname><given-names>D.</given-names></name>
<name><surname>Valerio</surname><given-names>M.</given-names></name>
<name><surname>Garcia-Vidal</surname><given-names>C.</given-names></name>
<name><surname>Falces-Romero</surname><given-names>I.</given-names></name>
<name><surname>Machado</surname><given-names>M.</given-names></name>
<name><surname>de la Villa</surname><given-names>S.</given-names></name>
<etal/>
</person-group><article-title>COVID-19&#x02013;associated pulmonary aspergillosis, March&#x02013;August 2020</article-title><source>Emerg. Infect. Dis.</source><year>2021</year><volume>27</volume><fpage>1077</fpage><pub-id pub-id-type="doi">10.3201/eid2704.204895</pub-id><pub-id pub-id-type="pmid">33539721</pub-id></element-citation></ref><ref id="B13-jof-07-00708"><label>13.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>John</surname><given-names>T.</given-names></name>
<name><surname>Jacob</surname><given-names>C.</given-names></name>
<name><surname>Kontoyiannis</surname><given-names>D.</given-names></name>
</person-group><article-title>When uncontrolled diabetes mellitus and severe COVID-19 converge: The perfect storm for mucormycosis</article-title><source>J. Fungi</source><year>2021</year><volume>7</volume><elocation-id>298</elocation-id><pub-id pub-id-type="doi">10.3390/jof7040298</pub-id><pub-id pub-id-type="pmid">33920755</pub-id></element-citation></ref><ref id="B14-jof-07-00708"><label>14.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Garg</surname><given-names>D.</given-names></name>
<name><surname>Muthu</surname><given-names>V.</given-names></name>
<name><surname>Sehgal</surname><given-names>I.S.</given-names></name>
<name><surname>Ramachandran</surname><given-names>R.</given-names></name>
<name><surname>Kaur</surname><given-names>H.</given-names></name>
<name><surname>Bhalla</surname><given-names>A.</given-names></name>
<name><surname>Puri</surname><given-names>G.D.</given-names></name>
<name><surname>Chakrabarti</surname><given-names>A.</given-names></name>
<name><surname>Agarwal</surname><given-names>R.</given-names></name>
</person-group><article-title>Coronavirus Disease (COVID-19) Associated Mucormycosis (CAM): Case Report and Systematic Review of Literature</article-title><source>Mycopathologia</source><year>2021</year><volume>186</volume><fpage>289</fpage><lpage>298</lpage><pub-id pub-id-type="doi">10.1007/s11046-021-00528-2</pub-id><pub-id pub-id-type="pmid">33544266</pub-id></element-citation></ref><ref id="B15-jof-07-00708"><label>15.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Skiada</surname><given-names>A.</given-names></name>
<name><surname>Pavleas</surname><given-names>I.</given-names></name>
<name><surname>Drogari-Apiranthitou</surname><given-names>M.</given-names></name>
</person-group><article-title>Epidemiology and diagnosis of mucormycosis: An update</article-title><source>J. Fungi</source><year>2020</year><volume>6</volume><elocation-id>265</elocation-id><pub-id pub-id-type="doi">10.3390/jof6040265</pub-id><pub-id pub-id-type="pmid">33147877</pub-id></element-citation></ref><ref id="B16-jof-07-00708"><label>16.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Koehler</surname><given-names>P.</given-names></name>
<name><surname>Bassetti</surname><given-names>M.</given-names></name>
<name><surname>Chakrabarti</surname><given-names>A.</given-names></name>
<name><surname>Chen</surname><given-names>S.C.A.</given-names></name>
<name><surname>Colombo</surname><given-names>A.L.</given-names></name>
<name><surname>Hoenigl</surname><given-names>M.</given-names></name>
<name><surname>Klimko</surname><given-names>N.</given-names></name>
<name><surname>Lass-Fl&#x000f6;rl</surname><given-names>C.</given-names></name>
<name><surname>Oladele</surname><given-names>R.O.</given-names></name>
<name><surname>Vinh</surname><given-names>D.C.</given-names></name>
<etal/>
</person-group><article-title>Defining and managing COVID-19-associated pulmonary aspergillosis: The 2020 ECMM/ISHAM consensus criteria for research and clinical guidance</article-title><source>Lancet Infect. Dis.</source><year>2021</year><volume>21</volume><fpage>e149</fpage><lpage>e162</lpage><pub-id pub-id-type="doi">10.1016/S1473-3099(20)30847-1</pub-id><pub-id pub-id-type="pmid">33333012</pub-id></element-citation></ref><ref id="B17-jof-07-00708"><label>17.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Guinea</surname><given-names>J.</given-names></name>
</person-group><article-title>Updated EUCAST Clinical Breakpoints against Aspergillus, Implications for the Clinical Microbiology Laboratory</article-title><source>J. Fungi</source><year>2020</year><volume>6</volume><elocation-id>343</elocation-id><pub-id pub-id-type="doi">10.3390/jof6040343</pub-id><pub-id pub-id-type="pmid">33291313</pub-id></element-citation></ref><ref id="B18-jof-07-00708"><label>18.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Van Arkel</surname><given-names>A.L.E.</given-names></name>
<name><surname>Rijpstra</surname><given-names>T.A.</given-names></name>
<name><surname>Belderbos</surname><given-names>H.N.A.</given-names></name>
<name><surname>van Wijngaarden</surname><given-names>P.</given-names></name>
<name><surname>Verweij</surname><given-names>P.E.</given-names></name>
<name><surname>Bentvelsen</surname><given-names>R.G.</given-names></name>
</person-group><article-title>COVID-19-associated pulmonary aspergillosis</article-title><source>Am. J. Respir. Crit. Care Med.</source><year>2020</year><volume>202</volume><fpage>132</fpage><lpage>135</lpage><pub-id pub-id-type="doi">10.1164/rccm.202004-1038LE</pub-id><pub-id pub-id-type="pmid">32396381</pub-id></element-citation></ref><ref id="B19-jof-07-00708"><label>19.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Nasir</surname><given-names>N.</given-names></name>
<name><surname>Farooqi</surname><given-names>J.</given-names></name>
<name><surname>Mahmood</surname><given-names>S.F.</given-names></name>
<name><surname>Jabeen</surname><given-names>K.</given-names></name>
</person-group><article-title>COVID-19-associated pulmonary aspergillosis (CAPA) in patients admitted with severe COVID-19 pneumonia: An observational study from Pakistan</article-title><source>Mycoses</source><year>2020</year><volume>63</volume><fpage>766</fpage><lpage>770</lpage><pub-id pub-id-type="doi">10.1111/myc.13135</pub-id><pub-id pub-id-type="pmid">32585069</pub-id></element-citation></ref><ref id="B20-jof-07-00708"><label>20.</label><element-citation publication-type="journal"><person-group person-group-type="author">
<name><surname>Hong</surname><given-names>B.S.</given-names></name>
<name><surname>Kim</surname><given-names>M.-R.</given-names></name>
</person-group><article-title>Interplays between human microbiota and microRNAs in COVID-19 pathogenesis: A literature review</article-title><source>Phys. Act. Nutr.</source><year>2021</year><volume>25</volume><fpage>1</fpage><lpage>7</lpage><pub-id pub-id-type="doi">10.20463/pan.2021.0008</pub-id><pub-id pub-id-type="pmid">34315200</pub-id></element-citation></ref></ref-list></back><floats-group><fig position="float" id="jof-07-00708-f001"><label>Figure 1</label><caption><p>Timeline of the case, highlighting the therapies administered, the respiratory support provided and the evolution of the pulmonary lesion (<bold>A</bold> chest CT scan at day 38 since diagnosis, <bold>B</bold> chest CT scan at day 51 since diagnosis, <bold>C</bold> chest CT scan at day 58 since diagnosis).</p></caption><graphic xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="jof-07-00708-g001" position="float"/></fig></floats-group></article>"""
with open("open.xml", "w") as xml_file:
    xml_file.write(xmlString)

In [9]:
xml_file = open("open.xml", "r")

'\n  <books>\n    <book id="bk103">\n      <author>Corets, Eva</author>\n      <title>Maeve Ascendant</title>\n    </book>\n    <book id="bk104">\n      <author>Corets, Eva</author>\n      <title>Oberon\'s Legacy</title>\n    </book>\n  </books>'

In [19]:
from unstructured.partition.xml import partition_xml

# Partition the XML document
partitioned_elements = partition_xml("open.xml")

print(partitioned_elements)


[<unstructured.documents.elements.Title object at 0x1064b9350>, <unstructured.documents.elements.Title object at 0x1477d7fd0>, <unstructured.documents.elements.Title object at 0x1477d8150>, <unstructured.documents.elements.Title object at 0x1477d8990>, <unstructured.documents.elements.Title object at 0x1477d8ad0>, <unstructured.documents.elements.Text object at 0x1477d8f10>, <unstructured.documents.elements.Title object at 0x1477d9190>, <unstructured.documents.elements.Text object at 0x1477d9210>, <unstructured.documents.elements.Text object at 0x1477d9350>, <unstructured.documents.elements.Text object at 0x1477d9710>, <unstructured.documents.elements.Text object at 0x1477d9b10>, <unstructured.documents.elements.Title object at 0x1477d9e10>, <unstructured.documents.elements.Title object at 0x1477da490>, <unstructured.documents.elements.Title object at 0x1477da850>, <unstructured.documents.elements.Title object at 0x1477dab50>, <unstructured.documents.elements.Text object at 0x1477daad0

In [20]:
len(partitioned_elements)

631

In [21]:
doc1_pdf_elements_categories = [el.category for el in partitioned_elements]
collections.Counter(doc1_pdf_elements_categories)


Counter({'UncategorizedText': 355,
         'Title': 235,
         'NarrativeText': 22,
         'EmailAddress': 13,
         'ListItem': 6})

In [17]:
print(partitioned_elements[0].text)
print(partitioned_elements[1].text)
print(partitioned_elements[2].text)
print(partitioned_elements[3].text)


Corets, Eva
Maeve Ascendant
Corets, Eva
Oberon's Legacy


# Utility functions


In [6]:
def partition_pdf_dld(
    filename,
    skip_infer_table_types=[],
    pdf_infer_table_structure=True,
):
    """
    Partitions a PDF file into its constituent elements using the Unstructured API.

    This function reads a PDF file, partitions it into its elements such as text, tables, and images,
    and returns these elements in a structured format. It allows for the inference of table structures
    within the PDF and can skip certain types of tables during the inference process.

    Args:
        filename (str): The path to the PDF file to be partitioned.
        skip_infer_table_types (list[str], optional): A list of table types to skip during table structure inference. Defaults to [].
        pdf_infer_table_structure (bool, optional): A flag indicating whether to infer table structures within the PDF. Defaults to True.

    Returns:
        list[dict]: A list of dictionaries, each representing an element extracted from the PDF file.
    """
    with open(filename, "rb") as f:
        files = shared.Files(
            content=f.read(),
            file_name=filename,
        )

    req = shared.PartitionParameters(
        files=files,
        strategy="hi_res",
        hi_res_model_name="yolox",
        pdf_infer_table_structure=pdf_infer_table_structure,
        skip_infer_table_types=skip_infer_table_types,
    )

    try:
        resp = client.general.partition(req)
        dld_elements = dict_to_elements(resp.elements)
    except SDKError as e:
        print(e)
    return dld_elements


In [7]:
def parse_and_display_table_html(
    table_html: str, print_html: bool = False, display_html: bool = True
) -> None:
    """
    Parses and displays the given HTML table string.

    Args:
        table_html (str): The HTML string representation of the table to be parsed and displayed.

    Returns:
        None
    """
    parser = etree.XMLParser(remove_blank_text=True)
    file_obj = StringIO(table_html)
    tree = etree.parse(file_obj, parser)
    if print_html:
        print(etree.tostring(tree, pretty_print=True).decode())
    if display_html:
        display(HTML(table_html))

In [8]:
def print_element_category_counts(elements: list) -> None:
    """
    Prints the count of each category in the given list of elements.

    Args:
        elements (List[Element]): The list of elements to count the categories for.

    Returns:
        None
    """
    categories = [el.category for el in elements]
    print(collections.Counter(categories))

# File paths


In [9]:
file_dir = Path.cwd().parents[1] / "data" / "pdf"
doc1 = file_dir / "DOA-vs-VKA-blood-clots.pdf"
doc2 = file_dir / "fimmu-14-1145145.pdf"


# Extract elements using different strategies


## 1) Fast strategy -- only text extraction from a text-based PDF, then content normalization


In [10]:
doc1_pdf_elements = partition_pdf(
    filename=doc1,
    strategy="fast",
)


In [11]:
doc1_pdf_elements


In [10]:
doc1_pdf_elements_categories = [el.category for el in doc1_pdf_elements]
collections.Counter(doc1_pdf_elements_categories)


Counter({'NarrativeText': 143,
         'UncategorizedText': 121,
         'Title': 114,
         'ListItem': 52})

## 2) Document-layout-detection (DLD) strategy -- extract text and layout from a mixed-format PDF


In [11]:
str(doc1)


'/Users/ducdo/Repo/Demo/data/pdf/DOA-vs-VKA-blood-clots.pdf'

In [ ]:
# doc1_elements = partition_pdf_dld(
#     filename=str(doc1), skip_infer_table_types=[], pdf_infer_table_structure=True
# )

In [25]:
# with open("DOA-vs-VKA-blood-clots_dld_elements.pkl", "wb") as f:
#     pickle.dump(doc1_elements, f)

with open("DOA-vs-VKA-blood-clots_dld_elements.pkl", "rb") as f:
    doc1_elements = pickle.load(f)


In [27]:
print(collections.Counter([el.category for el in doc1_elements]))

Counter({'Title': 437, 'UncategorizedText': 125, 'ListItem': 45, 'NarrativeText': 44, 'Header': 14, 'Footer': 4, 'FigureCaption': 1, 'Table': 1})


In [15]:
doc1_tables = [el for el in doc1_elements if el.category == "Table"]
# doc1_table_example = doc1_tables[0].text
# table_html = doc1_tables[0].metadata.text_as_html
parse_and_display_table_html(doc1_tables[0].metadata.text_as_html)

In [ ]:
# doc2_elements = partition_pdf_dld(
#     filename=str(doc2), skip_infer_table_types=[], pdf_infer_table_structure=True
# )
# with open("fimmu-14-1145145_dld_elements.pkl", "wb") as f:
#     pickle.dump(doc2_elements, f)

In [44]:
with open("fimmu-14-1145145_dld_elements.pkl", "rb") as f:
    doc2_elements = pickle.load(f)


In [30]:
doc2_tables = [el for el in doc2_elements if el.category == "Table"]
# doc2_table_example = doc2_tables[0].text
parse_and_display_table_html(doc2_tables[0].metadata.text_as_html)


Medicat,Mech,Perspective
Eculizumab,Complement 5 inhibitor,"For CAPS refractory to standard treatment, CAPS receiving kidney transplantation, acute TMA in patients with aPL-related nephropathy, APS during pregnancy, or pediatric CAPS (2, 33, 34)"
Rituximab,Type I anti-CD20 monoclonal antibody,"For thrombocytopenia, hemolytic anemia, or other aPL-mediated hematological and microthrombotic manifestations or noncriteria manifestations; an alternative option for CAPS which is refractory to standard treatment, refractory obstetric APS, and pediatric CAPS (2, 33, 35)"
Obinutuzumab,Type II anti-CD20 monoclonal antibody (B-cell depletion mainly via DCD),Alternative option for rituximab in APS (36)
Belimumab,BAFF/Blys inhibitor,"Potential treatment for aPL-positive patients, or primary APS with high thrombotic risk (37-39)"
Daratumumab,Anti-CD38 monoclonal antibody,"Potential treatment for refractory APS (40, 41)"
Zanubrutinib,BTK inhibitor,"Unclear, evidence still being collected"
Anti-TNF-o therapy,"Anti-TNF-o. monoclonal antibody: adalimumab, certolizumab",In refractory obstetric APS (42)


## Process elements


### Remove references


In [45]:
def remove_references(pdf_elements):
    reference_title = [
        el for el in pdf_elements if el.category == "Title" and "References" in el.text
    ][0]
    pdf_elements = [
        el for el in pdf_elements if el.metadata.parent_id != reference_title.id
    ]
    return pdf_elements


doc1_elements = remove_references(doc1_elements)
doc2_elements = remove_references(doc2_elements)

In [48]:
print(collections.Counter([el.category for el in doc2_elements]))


Counter({'NarrativeText': 177, 'Title': 41, 'Header': 13, 'Footer': 11, 'UncategorizedText': 7, 'Table': 1, 'Image': 1, 'FigureCaption': 1})


<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> Still need to do more processing to remove references</p>


## Create chunks


In [49]:
doc1_chunks = chunk_by_title(
    doc1_elements,
    combine_text_under_n_chars=100,
    max_characters=3000,
)

doc2_chunks = chunk_by_title(
    doc2_elements,
    combine_text_under_n_chars=100,
    max_characters=3000,
)


# Create embeddings


In [52]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores.pgvector import PGVector

In [60]:
documents = []
for chunk in doc2_chunks:
    metadata = {}
    metadata["source"] = str(doc2).split("/")[-1]
    documents.append(Document(page_content=chunk.text, metadata=metadata))

In [87]:
documents[0]

Document(metadata={'source': 'fimmu-14-1145145.pdf'}, page_content='?frontiers | Frontiers in Immunology\n\nTYPE Review PUBLISHED 19 May 2023 DOI 10.3389/fimmu.2023.1145145\n\n® Check for updates')

In [98]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]


# Upload embeddings


In [116]:
load_dotenv()


True

In [113]:
os.getenv("OPENAI_EMBEDDING_MODEL")


'text-embedding-3-small'

In [114]:
embeddings = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))


In [115]:
embeddings.model

'text-embedding-3-small'

In [117]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


In [118]:
import time

index_name = os.getenv("PINECONE_INDEX_NAME")

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)


In [126]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)
if index.describe_index_stats().total_vector_count > 0:
    print("Deleting existing vectors")
    vector_store.delete(delete_all=True)


In [127]:
vector_store.add_documents(documents=documents, ids=uuids)

['025d6442-38bb-40a5-b926-b37d372ca716',
 'a5bbd1f2-d9ef-4510-a9aa-017b67771482',
 'b917e9df-ce05-4d61-9844-0be5d06aaf60',
 '0f64c5a3-1af7-4c96-a398-14e8f1bf82ad',
 'c899814b-400a-4988-a53f-6660a012db0e',
 'd4870cdb-565c-4f2d-a507-a7f785f38de2',
 'd7a91249-a187-43a3-b8fb-b0d23a931214',
 '9906bf38-96e2-4f65-94a5-c62598a89156',
 '425499d7-f61c-4e0a-bc80-ca10dec1eb4c',
 '1af981b7-1359-4d01-aa27-389cb1fbb5cd',
 '3a7565b4-815b-478e-8cd5-8b0ea10bb7f3',
 'f7ca473a-d68d-4c01-8cb2-2d4527f99abe',
 'e3672b13-3cd5-4470-ad75-8a072b50eaf9',
 '8e0569a1-a04b-4cae-bc55-0a724941c1dd',
 '92fa2472-77c2-4281-91d0-c369e5a75f78',
 'ae94bdb8-a19d-4f8f-ba6c-892fa5f28663',
 '765518de-9899-48c4-aeaa-fba835dbfa01',
 '16764bbb-eaa7-4895-8255-df8f0a66b586',
 '5d2d67d4-bc01-4f98-a210-70fbd6ed062e',
 'bc41ba5a-671c-4750-a327-1e8e980d24f7',
 '151d89fa-9159-45e4-97a4-41806a579ac6',
 '86c3d229-1d60-47e9-8cba-f9ccd3697a9d',
 '15c44cbb-d62c-4f50-a581-bceab6f270b3',
 '2c6b0f52-96fb-4fe3-a361-e5f5a903c49e',
 '7950c973-9105-

In [129]:
retriever = vector_store.as_retriever(
    # search_type=os.getenv("RETRIEVER_SEARCH_TYPE"),
    search_kwargs={"k": 10},
)
print(retriever.invoke("What are some conclusions about anticoagulation"))

[Document(id='939fb9dc-9811-45a1-aa2f-91b9115ff26c', metadata={'source': 'fimmu-14-1145145.pdf'}, page_content='Conclusions\n\nAlthough anticoagulation remains the cornerstone of APS treatment, signiﬁcant attention is being drawn towards studying a number of biologics. It is important to note that biologics are primarily reserved for patients with refractory APS or CAPS. However, the accuracy of biologics make them a promising option for the development of optimal therapies for personalized medicine. Innovative biologic-focused therapeutic approaches are being investigated to treat APS, which may ultimately reduce mortality rates among individuals with APS.'), Document(id='d7a91249-a187-43a3-b8fb-b0d23a931214', metadata={'source': 'fimmu-14-1145145.pdf'}, page_content='Introduction\n\nAntiphospholipid syndrome (APS) is a systemic autoimmune disease, characterized by thrombosis or recurrent pregnancy morbidity, with the persistent presence of antiphospholipid antibodies (aPLs) (1, 2). A

In [130]:
retriever.invoke("{'query': 'anticoagulation conclusions'}")


[Document(id='939fb9dc-9811-45a1-aa2f-91b9115ff26c', metadata={'source': 'fimmu-14-1145145.pdf'}, page_content='Conclusions\n\nAlthough anticoagulation remains the cornerstone of APS treatment, signiﬁcant attention is being drawn towards studying a number of biologics. It is important to note that biologics are primarily reserved for patients with refractory APS or CAPS. However, the accuracy of biologics make them a promising option for the development of optimal therapies for personalized medicine. Innovative biologic-focused therapeutic approaches are being investigated to treat APS, which may ultimately reduce mortality rates among individuals with APS.'),
 Document(id='f907ed5c-4e2c-4881-81c1-7c29a2d6319b', metadata={'source': 'fimmu-14-1145145.pdf'}, page_content='References\n\n07\n\nfrontiersin.org\n\nYun et al.\n\n10.3389/ﬁmmu.2023.1145145\n\nantiphospholipid syndrome treatment trends. Lupus (2020) 29(12):1571–93. doi: 10.1177/0961203320950461\n\n43. Vega-Ostertag M, Liu X, Kwa